https://huggingface.co/spaces/qubvel-hf/albumentations-demo?transform=AdvancedBlur

Flip: Horizontal, Vertical
Rotation: Between -45° and +45°
Noise: Up to 1.8% of pixels

dataset/
    images/
        class1/
            img1.jpg
            img2.jpg
            ...
        class2/
            img3.jpg
            img4.jpg
            ...
        ...
    labels/
            img1.txt
            img2.txt
            img3.txt
            img4.txt
            ...


In [7]:
import os
import cv2
import shutil
import albumentations as A

# 設置路徑
images_path = r"C:\Users\Administrator\Desktop\test\images"
labels_path = r"C:\Users\Administrator\Desktop\test\labels"
augmented_images_path = r"C:\Users\Administrator\Desktop\test\aug_data\aug_images"
augmented_labels_path = r"C:\Users\Administrator\Desktop\test\aug_data\aug_labels"

# 創建增強後的目錄
os.makedirs(augmented_images_path, exist_ok=True)
os.makedirs(augmented_labels_path, exist_ok=True)

# 定義增強管道
transform_pipes = [
    A.Compose([
        A.HorizontalFlip(p=1.0),
    ], bbox_params=A.BboxParams(format='yolo', label_fields=['class_labels'])),
    
    A.Compose([
        A.VerticalFlip(p=1.0),
    ], bbox_params=A.BboxParams(format='yolo', label_fields=['class_labels'])),
    
    A.Compose([
        A.Rotate(limit=(-90, 90), p=1.0),
    ], bbox_params=A.BboxParams(format='yolo', label_fields=['class_labels'])),
    
    A.Compose([
        A.GaussNoise(var_limit=(10.0, 50.0), p=1.0),
    ], bbox_params=A.BboxParams(format='yolo', label_fields=['class_labels'])),
    
    A.Compose([
        A.Sharpen(alpha=(0.2, 0.5), lightness=(0.5, 1.0), p=1.0),
    ], bbox_params=A.BboxParams(format='yolo', label_fields=['class_labels'])),
    
    A.Compose([
        A.RandomBrightnessContrast(brightness_limit=(-0.2, 0.2), contrast_limit=(-0.2, 0.2), p=1.0),
    ], bbox_params=A.BboxParams(format='yolo', label_fields=['class_labels'])),

    A.Compose([
        A.Resize(
    height=300,  # int
    width=300,  # int
    interpolation=1,  # <class 'int'>
    always_apply=None,  # bool | None
    p=1.0,  # float
    ),], bbox_params=A.BboxParams(format='yolo', label_fields=['class_labels'])),

    
]

# 讀取標註文件
def read_yolo_labels(file_path):
    bboxes = []
    class_labels = []
    with open(file_path, 'r') as file:
        lines = file.readlines()
        for line in lines:
            parts = line.strip().split()
            if len(parts) != 5:
                print(f"Skipping line with incorrect format: {line.strip()}")
                continue
            try:
                class_id, x_center, y_center, width, height = map(float, parts)
                bboxes.append([x_center, y_center, width, height])
                class_labels.append(int(class_id))
            except ValueError:
                print(f"Error parsing line: {line.strip()}")
                continue
    return bboxes, class_labels

# 寫入標註文件
def write_yolo_labels(file_path, bboxes, class_labels):
    with open(file_path, 'w') as file:
        for bbox, class_label in zip(bboxes, class_labels):
            x_center, y_center, width, height = bbox
            file.write(f"{class_label} {x_center} {y_center} {width} {height}\n")

# 針對每個類別的圖像和標註進行增強
for class_name in os.listdir(images_path):
    class_images_path = os.path.join(images_path, class_name)
    
    if not os.path.isdir(class_images_path):
        continue
    
    class_augmented_images_path = os.path.join(augmented_images_path, class_name)
    os.makedirs(class_augmented_images_path, exist_ok=True)

    for image_name in os.listdir(class_images_path):
        image_path = os.path.join(class_images_path, image_name)
        label_path = os.path.join(labels_path, os.path.splitext(image_name)[0] + ".txt")
        
        image = cv2.imread(image_path)
        if image is None:
            continue
        
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        bboxes, class_labels = read_yolo_labels(label_path)

        # 確認標註數據格式是否正確
        if not bboxes or not class_labels:
            print(f"Invalid label data in {label_path}. Skipping...")
            continue

        # 複製原始圖像和標註
        original_image_dst = os.path.join(class_augmented_images_path, image_name)
        shutil.copy2(image_path, original_image_dst)
        
        original_label_dst = os.path.join(augmented_labels_path, os.path.splitext(image_name)[0] + ".txt")
        shutil.copy2(label_path, original_label_dst)

        # 對每個增強管道應用增強
        for i, transform in enumerate(transform_pipes):
            try:
                transformed = transform(image=image, bboxes=bboxes, class_labels=class_labels)
            except Exception as e:
                print(f"Error applying transform {i} on {image_name}: {e}")
                continue

            transformed_image = transformed['image']
            transformed_bboxes = transformed['bboxes']
            transformed_class_labels = transformed['class_labels']

            augmented_image_path = os.path.join(class_augmented_images_path, f"aug_{i}_{image_name}")
            cv2.imwrite(augmented_image_path, cv2.cvtColor(transformed_image, cv2.COLOR_RGB2BGR))

            augmented_label_path = os.path.join(augmented_labels_path, f"aug_{i}_{os.path.splitext(image_name)[0]}.txt")
            write_yolo_labels(augmented_label_path, transformed_bboxes, transformed_class_labels)

print("圖像及其標註增強完成並保存。")


圖像及其標註增強完成並保存。


## 步驟 1：安裝必要的 Python 包

pip install split-folders

## 步驟 2：準備你的數據集

In [2]:
import os
import splitfolders
import shutil

# 設置輸入資料夾和輸出資料夾
input_images_folder = r"C:\Users\Administrator\Desktop\test\aug_data\aug_images"
input_labels_folder = r"C:\Users\Administrator\Desktop\test\aug_data\aug_labels"
output_folder = r"C:\Users\Administrator\Desktop\test\split_data"

# 使用 splitfolders 將圖像分割成 train、val 和 test 資料夾
splitfolders.ratio(input_images_folder, output=output_folder, seed=1337, ratio=(.7, .2, .1), group_prefix=None)

# 確保輸出資料夾結構
for split in ['train', 'val', 'test']:
    os.makedirs(os.path.join(output_folder, split, 'images'), exist_ok=True)
    os.makedirs(os.path.join(output_folder, split, 'labels'), exist_ok=True)

# 定義函數來移動標註文件
def move_labels(src_labels_folder, dest_labels_folder, image_name):
    label_file = os.path.splitext(image_name)[0] + ".txt"
    src_label_path = os.path.join(src_labels_folder, label_file)
    if os.path.exists(src_label_path):
        dest_label_path = os.path.join(dest_labels_folder, label_file)
        shutil.copy2(src_label_path, dest_label_path)

# 移動圖像和標註文件
for split in ['train', 'val', 'test']:
    split_images_path = os.path.join(output_folder, split)
    split_labels_path = os.path.join(output_folder, split, 'labels')
    
    # 遍歷每個類別的資料夾
    for item in os.listdir(split_images_path):
        item_path = os.path.join(split_images_path, item)
        
        if os.path.isdir(item_path):
            # 移動圖像文件到統一的 images 資料夾
            for image_name in os.listdir(item_path):
                if image_name.endswith(('.jpg', '.jpeg', '.png')):
                    src_image_path = os.path.join(item_path, image_name)
                    dest_image_path = os.path.join(split_images_path, 'images', image_name)
                    shutil.move(src_image_path, dest_image_path)
                    
                    # 移動對應的標註文件
                    move_labels(input_labels_folder, split_labels_path, image_name)
            
            # 刪除空的類別資料夾
            if not os.listdir(item_path):
                os.rmdir(item_path)

print("圖像和標註文件已成功分割並整理完成。")


圖像和標註文件已成功分割並整理完成。
